## Demo 12/04

In [1]:
import lsdb
from lsdb.core.crossmatch.kdtree_match import KdTreeCrossmatch

### Issue [#592](https://github.com/astronomy-commons/hats/issues/592)

Reading through HTTP is slow because we check if the pointer is a directory or not and that downloads the entire file.

#### [Before fix](https://hbt.github.io/html-previewer.html?url=https%3A%2F%2Fgist.github.com%2Fhombit%2F8dc2afbcb8fb9ce4edf7b2579ff13053)

<img src="images/benchmark_before.png" width="1000">

Allows to pass a `is_dir` flag from HATS to avoid checking if pointer is a directory:

```python
# From LSDB, send the flag over to HATS
ddf = nd.NestedFrame.from_map(
    read_pixel,
    ordered_pixels,
    path_generator=config.path_generator,
    catalog_base_dir=catalog.catalog_base_dir,
    npix_suffix=npix_suffix,
    query_url_params=query_url_params,
    columns=config.columns,
    schema=catalog.schema,
    filters=config.filters,
    index_column=index_column,
    divisions=divisions,
    meta=dask_meta_schema,
    # Leaves are directories if `npix_suffix` == "/"
    is_dir=(npix_suffix == "/"),
    **config.kwargs,
)

...

# In HATS, only run the expensive isdir() operation if is_dir was not set
def read_parquet_file_to_pandas(
    file_pointer: str | Path | UPath, is_dir: bool | None = None, **kwargs
) -> npd.NestedFrame:
    """Reads parquet file(s) to a pandas DataFrame

    Parameters
    ----------
    file_pointer: str | Path | UPath
        File Pointer to a parquet file or a directory containing parquet files
    is_dir : bool | None
        If True, the pointer represents a pixel directory, otherwise, the pointer
        represents a file. In both cases there is no need to check the pointer's
        content type. If `is_dir` is None (default), this method will resort to
        `upath.is_dir()` to identify the type of pointer. Inferring the type for
        HTTP is particularly expensive because it requires downloading the contents
        of the pointer in its entirety.
    **kwargs
        Additional arguments to pass to pandas read_parquet method

    Returns
    -------
    NestedFrame
        Pandas DataFrame with the data from the parquet file(s)
    """
    if is_dir is None:
        is_dir = file_pointer.is_dir()
    ...
```

#### [After fix](https://hbt.github.io/html-previewer.html?url=https%3A%2F%2Fgist.github.com%2Fcamposandro%2Ff28bcfa2b3421cda2c510410060bde3a)

<img src="images/benchmark_after.png" width="1000">

### Issue [#946](https://github.com/astronomy-commons/lsdb/issues/946)

The crossmatch API accepted an algorithm class type `algorithm` and its arguments as `**kwargs`.

Users needed to read the docstrings to figure out the possible kwargs. IDEs wouldn't detect them either.

```python
def crossmatch(
    self,
    other: Catalog,
    suffixes: tuple[str, str] | None = None,
    algorithm: (
        type[AbstractCrossmatchAlgorithm] | BuiltInCrossmatchAlgorithm
    ) = BuiltInCrossmatchAlgorithm.KD_TREE,
    output_catalog_name: str | None = None,
    require_right_margin: bool = False,
    suffix_method: str | None = None,
    log_changes: bool = True,
    **kwargs,
) -> Catalog:
```

The new API specifies the default kwargs directly in the signature. A custom algorithm INSTANCE can still be provided.

```python
def crossmatch(
    self,
    other: Catalog,
    n_neighbors: int | None = None,
    radius_arcsec: float | None = None,
    min_radius_arcsec: float | None = None,
    algorithm: CrossmatchAlgorithm | None = None,
    output_catalog_name: str | None = None,
    require_right_margin: bool = False,
    suffixes: tuple[str, str] | None = None,
    suffix_method: str | None = None,
    log_changes: bool = True,
) -> Catalog:
```

In [2]:
ztf = lsdb.open_catalog('s3://ipac-irsa-ztf/contributed/dr23/objects/hats')
gaia = lsdb.open_catalog('s3://stpubdata/gaia/gaia_dr3/public/hats')

The user does not need to instantiate a `KdTreeCrossmatch` object, they can use the explicitly defined kwargs:

In [3]:
ztf.crossmatch(gaia, n_neighbors=3, radius_arcsec=5)

/Users/scampos/lsdb/src/lsdb/catalog/catalog.py:307: FutureWarning: The default suffix behavior will change from applying suffixes to all columns to only applying suffixes to overlapping columns in a future release.To maintain the current behavior, explicitly set `suffix_method='all_columns'`. To change to the new behavior, set `suffix_method='overlapping_columns'`.
  warnings.warn(


,oid_ZTF_DR23_Objects,ra_ZTF_DR23_Objects,dec_ZTF_DR23_Objects,filtercode_ZTF_DR23_Objects,medianmag_ZTF_DR23_Objects,medmagerr_ZTF_DR23_Objects,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,parallax_over_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,ra_dec_corr_gaia,ra_parallax_corr_gaia,ra_pmra_corr_gaia,ra_pmdec_corr_gaia,dec_parallax_corr_gaia,dec_pmra_corr_gaia,dec_pmdec_corr_gaia,parallax_pmra_corr_gaia,parallax_pmdec_corr_gaia,pmra_pmdec_corr_gaia,astrometric_n_obs_al_gaia,astrometric_n_obs_ac_gaia,astrometric_n_good_obs_al_gaia,astrometric_n_bad_obs_al_gaia,astrometric_gof_al_gaia,astrometric_chi2_al_gaia,astrometric_excess_noise_gaia,astrometric_excess_noise_sig_gaia,astrometric_params_solved_gaia,astrometric_primary_flag_gaia,nu_eff_used_in_astrometry_gaia,pseudocolour_gaia,pseudocolour_error_gaia,ra_pseudocolour_corr_gaia,dec_pseudocolour_corr_gaia,parallax_pseudocolour_corr_gaia,pmra_pseudocolour_corr_gaia,pmdec_pseudocolour_corr_gaia,astrometric_matched_transits_gaia,visibility_periods_used_gaia,astrometric_sigma5d_max_gaia,matched_transits_gaia,new_matched_transits_gaia,matched_transits_removed_gaia,ipd_gof_harmonic_amplitude_gaia,ipd_gof_harmonic_phase_gaia,ipd_frac_multi_peak_gaia,ipd_frac_odd_win_gaia,ruwe_gaia,scan_direction_strength_k1_gaia,scan_direction_strength_k2_gaia,scan_direction_strength_k3_gaia,scan_direction_strength_k4_gaia,scan_direction_mean_k1_gaia,scan_direction_mean_k2_gaia,scan_direction_mean_k3_gaia,scan_direction_mean_k4_gaia,duplicated_source_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_flux_over_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_flux_over_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_flux_over_error_gaia,phot_rp_mean_mag_gaia,phot_bp_rp_excess_factor_gaia,phot_bp_n_contaminated_transits_gaia,phot_bp_n_blended_transits_gaia,phot_rp_n_contaminated_transits_gaia,phot_rp_n_blended_transits_gaia,phot_proc_mode_gaia,bp_rp_gaia,bp_g_gaia,g_rp_gaia,radial_velocity_gaia,radial_velocity_error_gaia,rv_method_used_gaia,rv_nb_transits_gaia,rv_nb_deblended_transits_gaia,rv_visibility_periods_used_gaia,rv_expected_sig_to_noise_gaia,rv_renormalised_gof_gaia,rv_chisq_pvalue_gaia,rv_time_duration_gaia,rv_amplitude_robust_gaia,rv_template_teff_gaia,rv_template_logg_gaia,rv_template_fe_h_gaia,rv_atm_param_origin_gaia,vbroad_gaia,vbroad_error_gaia,vbroad_nb_transits_gaia,grvs_mag_gaia,grvs_mag_error_gaia,grvs_mag_nb_transits_gaia,rvs_spec_sig_to_noise_gaia,phot_variable_flag_gaia,l_gaia,b_gaia,ecl_lon_gaia,ecl_lat_gaia,in_qso_candidates_gaia,in_galaxy_candidates_gaia,non_single_star_gaia,has_xp_continuous_gaia,has_xp_sampled_gaia,has_rvs_gaia,has_epoch_photometry_gaia,has_epoch_rv_gaia,has_mcmc_gspphot_gaia,has_mcmc_msc_gaia,in_andromeda_survey_gaia,classprob_dsc_combmod_quasar_gaia,classprob_dsc_combmod_galaxy_gaia,classprob_dsc_combmod_star_gaia,teff_gspphot_gaia,teff_gspphot_lower_gaia,teff_gspphot_upper_gaia,logg_gspphot_gaia,logg_gspphot_lower_gaia,logg_gspphot_upper_gaia,mh_gspphot_gaia,mh_gspphot_lower_gaia,mh_gspphot_upper_gaia,distance_gspphot_gaia,distance_gspphot_lower_gaia,distance_gspphot_upper_gaia,azero_gspphot_gaia,azero_gspphot_lower_gaia,azero_gspphot_upper_gaia,ag_gspphot_gaia,ag_gspphot_lower_gaia,ag_gspphot_upper_gaia,ebpminrp_gspphot_gaia,ebpminrp_gspphot_lower_gaia,ebpminrp_gspphot_upper_gaia,libname_gspphot_gaia,_dist_arcsec
npartitions=2298,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 3, Pixel: 0",int64[pyarrow],double[pyarrow],double[pyarrow],string[pyarrow],float[pyarrow],float[pyarrow],int64[pyarrow],string[pyarrow],int64[pyarrow],

If a custom algorithm defines attributes with the same name as the default kwargs, we need to make sure they do not conflict: 

In [4]:
class MyAlgorithm(KdTreeCrossmatch):
    n_neighbors: int = 1
ztf.crossmatch(gaia, algorithm=MyAlgorithm(), n_neighbors=2)

ValueError: If you specify `algorithm`, do not set ['n_neighbors']

In this case, just specify the kwarg as an attribute to the algorithm:

In [5]:
ztf.crossmatch(gaia, algorithm=MyAlgorithm(n_neighbors=2))

/Users/scampos/lsdb/src/lsdb/catalog/catalog.py:307: FutureWarning: The default suffix behavior will change from applying suffixes to all columns to only applying suffixes to overlapping columns in a future release.To maintain the current behavior, explicitly set `suffix_method='all_columns'`. To change to the new behavior, set `suffix_method='overlapping_columns'`.
  warnings.warn(


,oid_ZTF_DR23_Objects,ra_ZTF_DR23_Objects,dec_ZTF_DR23_Objects,filtercode_ZTF_DR23_Objects,medianmag_ZTF_DR23_Objects,medmagerr_ZTF_DR23_Objects,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,parallax_over_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,ra_dec_corr_gaia,ra_parallax_corr_gaia,ra_pmra_corr_gaia,ra_pmdec_corr_gaia,dec_parallax_corr_gaia,dec_pmra_corr_gaia,dec_pmdec_corr_gaia,parallax_pmra_corr_gaia,parallax_pmdec_corr_gaia,pmra_pmdec_corr_gaia,astrometric_n_obs_al_gaia,astrometric_n_obs_ac_gaia,astrometric_n_good_obs_al_gaia,astrometric_n_bad_obs_al_gaia,astrometric_gof_al_gaia,astrometric_chi2_al_gaia,astrometric_excess_noise_gaia,astrometric_excess_noise_sig_gaia,astrometric_params_solved_gaia,astrometric_primary_flag_gaia,nu_eff_used_in_astrometry_gaia,pseudocolour_gaia,pseudocolour_error_gaia,ra_pseudocolour_corr_gaia,dec_pseudocolour_corr_gaia,parallax_pseudocolour_corr_gaia,pmra_pseudocolour_corr_gaia,pmdec_pseudocolour_corr_gaia,astrometric_matched_transits_gaia,visibility_periods_used_gaia,astrometric_sigma5d_max_gaia,matched_transits_gaia,new_matched_transits_gaia,matched_transits_removed_gaia,ipd_gof_harmonic_amplitude_gaia,ipd_gof_harmonic_phase_gaia,ipd_frac_multi_peak_gaia,ipd_frac_odd_win_gaia,ruwe_gaia,scan_direction_strength_k1_gaia,scan_direction_strength_k2_gaia,scan_direction_strength_k3_gaia,scan_direction_strength_k4_gaia,scan_direction_mean_k1_gaia,scan_direction_mean_k2_gaia,scan_direction_mean_k3_gaia,scan_direction_mean_k4_gaia,duplicated_source_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_flux_over_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_flux_over_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_flux_over_error_gaia,phot_rp_mean_mag_gaia,phot_bp_rp_excess_factor_gaia,phot_bp_n_contaminated_transits_gaia,phot_bp_n_blended_transits_gaia,phot_rp_n_contaminated_transits_gaia,phot_rp_n_blended_transits_gaia,phot_proc_mode_gaia,bp_rp_gaia,bp_g_gaia,g_rp_gaia,radial_velocity_gaia,radial_velocity_error_gaia,rv_method_used_gaia,rv_nb_transits_gaia,rv_nb_deblended_transits_gaia,rv_visibility_periods_used_gaia,rv_expected_sig_to_noise_gaia,rv_renormalised_gof_gaia,rv_chisq_pvalue_gaia,rv_time_duration_gaia,rv_amplitude_robust_gaia,rv_template_teff_gaia,rv_template_logg_gaia,rv_template_fe_h_gaia,rv_atm_param_origin_gaia,vbroad_gaia,vbroad_error_gaia,vbroad_nb_transits_gaia,grvs_mag_gaia,grvs_mag_error_gaia,grvs_mag_nb_transits_gaia,rvs_spec_sig_to_noise_gaia,phot_variable_flag_gaia,l_gaia,b_gaia,ecl_lon_gaia,ecl_lat_gaia,in_qso_candidates_gaia,in_galaxy_candidates_gaia,non_single_star_gaia,has_xp_continuous_gaia,has_xp_sampled_gaia,has_rvs_gaia,has_epoch_photometry_gaia,has_epoch_rv_gaia,has_mcmc_gspphot_gaia,has_mcmc_msc_gaia,in_andromeda_survey_gaia,classprob_dsc_combmod_quasar_gaia,classprob_dsc_combmod_galaxy_gaia,classprob_dsc_combmod_star_gaia,teff_gspphot_gaia,teff_gspphot_lower_gaia,teff_gspphot_upper_gaia,logg_gspphot_gaia,logg_gspphot_lower_gaia,logg_gspphot_upper_gaia,mh_gspphot_gaia,mh_gspphot_lower_gaia,mh_gspphot_upper_gaia,distance_gspphot_gaia,distance_gspphot_lower_gaia,distance_gspphot_upper_gaia,azero_gspphot_gaia,azero_gspphot_lower_gaia,azero_gspphot_upper_gaia,ag_gspphot_gaia,ag_gspphot_lower_gaia,ag_gspphot_upper_gaia,ebpminrp_gspphot_gaia,ebpminrp_gspphot_lower_gaia,ebpminrp_gspphot_upper_gaia,libname_gspphot_gaia,_dist_arcsec
npartitions=2298,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 3, Pixel: 0",int64[pyarrow],double[pyarrow],double[pyarrow],string[pyarrow],float[pyarrow],float[pyarrow],int64[pyarrow],string[pyarrow],int64[pyarrow],

### Issues [#831](https://github.com/astronomy-commons/lsdb/issues/831) / [#950](https://github.com/astronomy-commons/lsdb/issues/950)

Both `open_catalog` and `read_hats` were returning type `Dataset`. However:

- The catalogs we're opening are actually `HealpixDataset` or `Catalog`. 
- Mypy isn't able to infer their correct types.
- IDEs don't provide a full list of available methods for autocomplete.

To overcome this, `open_catalog` now only opens `Catalog`, `read_hats` opens all.

In [6]:
lsdb.open_catalog('s3://stpubdata/panstarrs/ps1/public/hats/otmo_10arcs')

TypeError: To load auxiliary datasets please use `lsdb.read_hats()`

<img src="images/autocomplete_object.png" width=800/>

In [7]:
lsdb.read_hats('s3://stpubdata/panstarrs/ps1/public/hats/otmo_10arcs')

,objName,objNameHMS,objAltName1,objAltName2,objAltName3,objPopularName,objID,uniquePspsOBid,ippObjID,surveyID,htmID,zoneID,tessID,projectionID,skyCellID,randomID,batchID,dvoRegionID,processingVersion,objInfoFlag,astrometryCorrectionFlag,qualityFlag,raStack,decStack,raStackErr,decStackErr,raMean,decMean,raMeanErr,decMeanErr,pmra,pmdec,pmraErr,pmdecErr,epochMean,posMeanChisq,cx,cy,cz,lambda,beta,l,b,nStackObjectRows,nStackDetections,nDetections,ng,nr,ni,nz,ny,gQfPerfect,gMeanPSFMag,gMeanPSFMagErr,gMeanPSFMagStd,gMeanPSFMagNpt,gMeanPSFMagMin,gMeanPSFMagMax,gMeanKronMag,gMeanKronMagErr,gMeanKronMagStd,gMeanKronMagNpt,gMeanApMag,gMeanApMagErr,gMeanApMagStd,gMeanApMagNpt,gFlags,rQfPerfect,rMeanPSFMag,rMeanPSFMagErr,rMeanPSFMagStd,rMeanPSFMagNpt,rMeanPSFMagMin,rMeanPSFMagMax,rMeanKronMag,rMeanKronMagErr,rMeanKronMagStd,rMeanKronMagNpt,rMeanApMag,rMeanApMagErr,rMeanApMagStd,rMeanApMagNpt,rFlags,iQfPerfect,iMeanPSFMag,iMeanPSFMagErr,iMeanPSFMagStd,iMeanPSFMagNpt,iMeanPSFMagMin,iMeanPSFMagMax,iMeanKronMag,iMeanKronMagErr,iMeanKronMagStd,iMeanKronMagNpt,iMeanApMag,iMeanApMagErr,iMeanApMagStd,iMeanApMagNpt,iFlags,zQfPerfect,zMeanPSFMag,zMeanPSFMagErr,zMeanPSFMagStd,zMeanPSFMagNpt,zMeanPSFMagMin,zMeanPSFMagMax,zMeanKronMag,zMeanKronMagErr,zMeanKronMagStd,zMeanKronMagNpt,zMeanApMag,zMeanApMagErr,zMeanApMagStd,zMeanApMagNpt,zFlags,yQfPerfect,yMeanPSFMag,yMeanPSFMagErr,yMeanPSFMagStd,yMeanPSFMagNpt,yMeanPSFMagMin,yMeanPSFMagMax,yMeanKronMag,yMeanKronMagErr,yMeanKronMagStd,yMeanKronMagNpt,yMeanApMag,yMeanApMagErr,yMeanApMagStd,yMeanApMagNpt,yFlags,Norder,Dir,Npix,margin_Norder,margin_Dir,margin_Npix
npartitions=27297,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 5, Pixel: 0",string[pyarrow],string[pyarrow],string[pyarrow],string[pyarrow],string[pyarrow],string[pyarrow],int64[pyarrow],int64[pyarrow],int64[pyarrow],int16[pyarrow],int64[pyarrow],int32[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],double[pyarrow],int64[pyarrow],int32[pyarrow],int16[pyarrow],int32[pyarrow],int32[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int32[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int32[pyarrow],int8[pyarrow],int64[pyarrow],int64[pyarrow],uint8[pyarrow],uint64[pyarrow],uint64[pyarrow]
"Order: 5, Pixel: 1",...,...,...,...,...,...,...,...,...,...,...,...,.

<img src="images/autocomplete_aux.png" width=800/>